# **AT&T Spam Detector**




Contents
--------
1. [Data loading and preprocessing](#loading)
2. [Preliminary data analysis](#eda)
3. [Benchmarking with a logistic regression](#logreg)
4. [Text processing](#text_processing)
4. [Conclusion and perspectives](#conclusion)



## <a id="loading"></a> Data loading and preprocessing

The dataset consists of 5574 entries, many of which have issues. Here is a non-exhaustive list:
- The file doesn't decode as `UTF-8`. It must be decoded as `latin-1` or `mac-roman`.
- Some rows are ill-formed (eg line 101 of the csv: `ham""",,,`)
- Most rows have trailing commas
- Some messages are anclosed in (possibly many) double quote characters
- Single and double quotes are escaped with backslashes
- Some spurious characters are present (eg `å` characters that systematically prepend pound symbols `£`)

Fortunately, the original version of the dataset can be found [here](https://archive.ics.uci.edu/dataset/228/sms+spam+collection). Its entries are much cleaner than those of our dataset. We can use them as a comparison to setup the cleaning and processing for our dataset:
- Strip the trailing commas
- Completely remove the double quotes `"`
- Convert backslashes + single quote `\'` into single quote `'`
- Convert backslashes `\` into double quotes `"`
- Remove the spurious characters `å`

The implementation of this pipeline is given below.

In [ ]:
########## Loading and preprocessing the project dataset ##########
with open('./spam.csv', 'rt', encoding='latin-1') as f:
    f.readline() # remove header
    raw_data = [row.strip().split(',', 1) for row in f.readlines()]

## set target
is_spam = np.array([True if row[0] == 'spam' else False for row in raw_data])

## process and clean messages
messages = [row[1].strip(',') \
                  .replace('"', '') \
                  .replace("\\'", "'") \
                  .replace('\\', '"') \
                  .replace('å', '')
            for row in raw_data]

However, some encoding issues persist with some problematic characters still present in messages. To proceed with the project we will rather use this original dataset as the starting point.

In [ ]:
########## Loading and preprocessing the original dataset ##########
with open('./SMSSpamCollection', 'rt', encoding='utf-8') as f:
    raw_data = [row.strip().split('\t', 1) for row in f.readlines()]

is_spam = np.array([True if row[0] == 'spam' else False for row in raw_data])
messages = np.array([row[1] for row in raw_data], dtype=object)

df = pd.DataFrame({'message': messages, 'is_spam': is_spam})
df.describe()

Our dataset seems to have duplicates. We choose not to remove them, they certainly represent frequent messages for which making a classification error should be penalized more.

## <a id="eda"></a> Preliminary data analysis

The messages are quite complex, mixing  uppercase, lowercase, puctuation, alphanumeric and other exotic characters. Training a model to the data would certainly require some form of text normalization. However, this might cause a loss of information.

To get some insights about what makes a message a spam and make visualizations, we will reduce the messages to a few descriptive quantities:
- `msg_len`, the length of the message;
- `chr_caps`, the number of capitallized characters;
- `chr_digit`, the number of digit characters in the message;
- `chr_punct`, the number of punctuation characters in the message;
- `max_wd_len`, he length of the longest character string (split by whitespaces);
- `lex_money`, the number of some money-related words and characters: `'$'`, `'£'`, `'€'`, `'cash'`, `'free'`, `'price'` and `'prize'`. The presence of these tokens indicates that the message talks about money (after all, spamming is about getting money from people). We could also consider adding words related to the lexical field of sex.
- `caps_first`, the number of consecutive capitallized characters at the beginning of the message.

For the purpose of data visualization, we remove the duplicates in the dataset.

In [ ]:
def count_tokens(msgs: pd.DataFrame, tokens: list['str'])-> pd.DataFrame:
    """
    Count the number of tokens in the token list from the casefolded messages.
    """
    regex = '|'.join(tokens)
    return msgs.apply(lambda x: len(re.findall(regex, x.casefold())))
    

def count_first_caps(s: str)-> int:
    """
    Count the number of capitallized letters at the beginning of a string.
    """
    n = 0
    for c in s:
        if not c.isupper():
            return n
        n += 1
    return n


def build_features(msgs: pd.DataFrame)-> pd.DataFrame:
    """
    Extract quantitative features from a series of messages.
    """
    data = {
        'msg_len': msgs.apply(len),
        'chr_caps': msgs.apply(lambda s: sum(c.isupper() for c in s)),
        'chr_digit': msgs.apply(lambda s: sum(c.isdigit() for c in s)),
        'chr_punct': msgs.apply(lambda s: sum(c in punctuation for c in s)),
        'max_wd_len': msgs.apply(lambda s: max(len(x) for x in s.split(' '))),
        'lex_money': count_tokens(msgs, [r'\$', '£', '€', 'free', 'cash', 'price', 'prize']),
        'caps_first': msgs.apply(count_first_caps)
        }
    return pd.DataFrame.from_dict(data)

In [ ]:
## Prepare dataset from extracted features
df_ = df.drop_duplicates()
X = build_features(df_['message'])
y = df_['is_spam']

In [ ]:
X_spam = X.loc[y]
X_spam.describe().applymap(lambda x: f"{x:.2f}")

In [ ]:
X_ham = X.loc[~y]
X_ham.describe().applymap(lambda x: f"{x:.2f}")

Some of our derived features are actually quite discriminating for the spam/ham nature of the messages.
- The number of digits is clearly the most discriminating feature with a mean of 15 for spams vs 0.3 for hams. 75 % of spams have more than 10 digits while less than 25% have a single digit.
- The presence of money-related words 'free', 'cash' and currency characters is also discriminative of spams, although many spams do not have any.
- Spams tend to also have larger message lengths, capitallized characters and punctuation characters. However, there is more overlap between the two categories in this case.

In [ ]:
v1, v2 = 'chr_digit', 'max_wd_len'

## number of hams with given values of `v1` and `v2`
count_ham = X_ham.loc[:, [v1, v2]].value_counts()
x_ham, y_ham = count_ham.index.to_frame().to_numpy().T

## number of spams with given values of `v1` and `v2`
count_spam = X_spam.loc[:, [v1, v2]].value_counts()
x_spam, y_spam = count_spam.index.to_frame().to_numpy().T

In [ ]:
fig1, ax1 = plt.subplots(
    nrows=1, ncols=1, figsize=(5.5, 4.8), dpi=200,
    gridspec_kw={'left': 0.12, 'right': 0.92, 'top': 0.9, 'bottom': 0.10}
)
fig1.suptitle('Figure 1: Scatter plot of messages characteristics',
              x=0.02, ha='left')

sc_ham = ax1.scatter(x_ham, y_ham, count_ham+2,
                     c='blue', alpha=0.65, marker='o', linewidths=0,
                     label='hams')
sc_spam = ax1.scatter(x_spam, y_spam, count_spam+2,
                      c='red', alpha=0.7, marker='o', linewidths=0,
                      label='spams')

ax1.set_xlim(-2, 50)
ax1.set_xticks(np.arange(5, 50, 5), minor=True)
ax1.set_xlabel('Number of digits `chr_digits`')

ax1.set_ylim(-0.5, 60)
ax1.set_ylabel('Max word length `max_wd_len`', labelpad=7)

ax1.grid(visible=True, which='major', linewidth=0.2)
ax1.grid(visible=True, which='minor', linewidth=0.1)

legend = ax1.legend()
legend.legend_handles[0]._sizes = [30]
legend.legend_handles[1]._sizes = [30]

plt.show()

We show on figure 1 a scatter plot of the number of digits and max word length for both hams (blue) and spams (red). The surface of the points is proportional to the number of messages (with an offset of 2 to facilitate vizualization). Most hams have few digits and are made of short words. We expect that a simple linear model in our engineered feature space will perform very well.

## <a id="logreg"></a> Benchmarking with a logistic regression

We set up a simple logistic regression here as a benchmark for comparison with more advanced deep learning methods.

ChatGPT prompt:

I need yout to 

## <a id="perpectives"></a>Perspectives

Possible extensions with more data/more precise data.

- Zipcode : do people prfer those who live closer
- Sex orientation